In [ ]:
!pip install -q -U trl transformers accelerate  git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

In [ ]:
!pip install streamlit
# !pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


## Front End Formation


In [ ]:
%%writefile app.py
import streamlit as st

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ArmurAI/solana-smart-contract-auditor"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def maker(code):
    eg = ''' you are an assistant State which of the popular vulnerabilities  are present in the following smart contract
    ### Human :
    {}
    ### Assistant:
    '''
    text = eg.format(code)
    device = "cuda:0"

    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=150)
    otpt = (tokenizer.decode(outputs[0], skip_special_tokens=True))
    final = otpt[otpt.index("### Assistant:")+15:]
    return final

def main():
  user_input = st.text_area("Enter your text here:", height=300)

  if user_input:
    output = maker(user_input)

    st.subheader("Output:")
    st.text_area("Result:", output, height=200)
  else:
    # Optionally display a message to encourage input
    st.info("Please enter some text to generate output.")

if __name__ == "__main__":
  main()

Writing app.py


In [ ]:
!nohub streamlit run app.py &

/bin/bash: line 1: nohub: command not found


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.234.58.37
npx: installed 22 in 2.937s
your url is: https://gold-rivers-spend.loca.lt


## Inference

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ArmurAI/solana-smart-contract-auditor"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

adapter_config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
eg = ''' you are an assistant State which of the popular vulnerabilities  are present in the following smart contract
### Human :
use solana_program::{
    account_info::{next_account_info, AccountInfo},
    entrypoint,
    entrypoint::ProgramResult,
    msg,
    pubkey::Pubkey,
};

entrypoint!(process_instruction);

pub fn process_instruction(
    program_id: &Pubkey,
    accounts: &[AccountInfo],
    instruction_data: &[u8],
) -> ProgramResult {
    let accounts_iter = &mut accounts.iter();
    let payer = next_account_info(accounts_iter)?;
    let receiver = next_account_info(accounts_iter)?;

    let amount = instruction_data[0] as u64;

    **payer.try_borrow_mut_lamports()? -= amount;
    **receiver.try_borrow_mut_lamports()? += amount;

    msg!("Transferred {} lamports from payer to receiver", amount);

    Ok(())
}
### Assistant:
'''
text = eg
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
otpt = (tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\n The Vulnerabilities are :")
final = otpt[otpt.index("### Assistant:")+15:]
print(otpt[otpt.index("### Assistant:")+15:])



## OUTPUT

The Vulnerabilities are :
The following popular vulnerabilities are present in the given smart contract:

1. Integer Overflow: The contract uses the `as` operator to convert the first byte of the instruction data to a `u64` type. This can cause integer overflow if the value of the first byte is greater than the maximum value that can be represented by
a `u64`. This can lead to unexpected behavior in the contract.
2. Unchecked Return: The contract calls the `next_account_info` function without checking its return value. This can cause the contract to panic if the function returns an error.
3. Mutable State: The contract uses mutable state (`payer.try_borrow_mut_lamports()?`) which can cause race conditions and other issues if not handled correctly. It is generally recommended to use immutable state where possible.
4. Lack of Error Handling: The contract does not handle errors returned by the `

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 The Vulnerabilities are :
The following popular vulnerabilities are present in the given smart contract:

1. Integer Overflow: The contract uses the `as` operator to convert the first byte of the instruction data to a `u64` type. This can cause integer overflow if the value of the first byte is greater than the maximum value that can be represented by a `u64`. This can lead to unexpected behavior in the contract.
2. Unchecked Return: The contract calls the `next_account_info` function without checking its return value. This can cause the contract to panic if the function returns an error.
3. Mutable State: The contract uses mutable state (`payer.try_borrow_mut_lamports()?`) which can cause race conditions and other issues if not handled correctly. It is generally recommended to use immutable state where possible.
4. Lack of Error Handling: The contract does not handle errors returned by the `
